In [1]:

ngram_length = 3
dstype = 'dbp' 
mname = 'debertaV3'

In [2]:
from colorama import Fore, Style

import os

In [3]:
path = f"/home/bhairavi/om/om3/{dstype}/{ngram_length}grams_{mname}/"
os.makedirs(path, exist_ok=True)

print("Directory created or already exists.")
 

file_path = path + f'{dstype}_{ngram_length}keys.csv' 
print(Fore.YELLOW,"csv_filePATH--->",file_path)


filepath_full = path + f'{dstype}_{ngram_length}top5.csv' 
print(Fore.YELLOW,"QUE_filePATH--->",filepath_full)
 
 
 
modelpath = f"/home/bhairavi/om/om5/{dstype}/{mname}_{dstype}"

 

print(Fore.YELLOW,'modelPATH--->',modelpath)
 


Directory created or already exists.
 csv_filePATH---> /home/bhairavi/om/om3/dbp/3grams_debertaV3/dbp_3keys.csv
 QUE_filePATH---> /home/bhairavi/om/om3/dbp/3grams_debertaV3/dbp_3top5.csv
 modelPATH---> /home/bhairavi/om/om5/dbp/debertaV3_dbp


In [4]:

 
 
import torch  

torch.cuda.empty_cache()  
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BertForSequenceClassification, BertTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
 


# %%
from collections import defaultdict
import string
 
import torch.nn.functional as F
from nltk.util import ngrams

import numpy as np  
  


/home/bhairavi/om/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

# %%
from datasets import load_dataset

dataset = load_dataset("DeveloperOats/DBPedia_Classes", name='default' )


# %%
dataset


# %%
dataset

# %%
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
validation_df = pd.DataFrame(dataset['validation'])

# %%
train_df['split'] = 'train'

test_df['split'] = 'test'

validation_df['split'] = 'validation'

# %%
df = pd.concat([train_df, test_df, validation_df], ignore_index=True)

# %%
df.shape

# %%
df.columns

# %%
 

# %%
df['l3'].nunique()

# %%


# %%

# %%
df['label'] = df['l3']


# %%

# %%
from sklearn.preprocessing import LabelEncoder

# %%
le = LabelEncoder()
df['target'] = le.fit_transform(df['label'])


# %%

# %%

# %%
df.columns

# %%


# %%
df.shape

# %%


# %%

  
 
 
 

# %%
numlabel = df['target'].nunique()
numlabel


# %%


# %%

# %%
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = "cuda"  # the device to load the model onto

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(modelpath)
model = AutoModelForSequenceClassification.from_pretrained(modelpath, num_labels=numlabel)

# Move the model to the specified device
model.to(device)


# %%


# %%
# df['token_length'] = df['text'].apply(lambda x: len(tokenizer.tokenize(x)))

# # Calculate the maximum token length
# max_length = df['token_length'].max()
# Calculate the token length using the split method
df['token_length'] = df['text'].apply(lambda x: len(x.split()))

# Calculate the maximum token length
max_length = df['token_length'].max()



# %%
max_length

# %%

# Calculate the next maximum token length
next_max_token_length = df['token_length'].nlargest(2).iloc[1] 


# Calculate the average token length
average_token_length = df['token_length'].mean()

# Display the results
print(f"Maximum token length: {max_length}")
print(f"Next maximum token length: {next_max_token_length}") 
print(f"Average token length: {average_token_length:.2f}")

# %%
min(df['token_length'])


# %%

# %%
fdf = df[df['token_length'] == 5]

# %%
fdf

# %%
df = df[df['token_length'] >= 5]

# %%
df.shape
 


# %%
df.columns

# %%
train_df = df[df['split'] == 'train'].drop(columns=['split'])

test_df = df[df['split'] == 'test'].drop(columns=['split'])

val_df = df[df['split'] == 'validation'].drop(columns=['split'])

# %%
train_df.shape, test_df.shape, val_df.shape

# %%

def tokenize_and_format(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=max_length)
    tokenized_inputs['label'] = list(map(int, examples['target']))
    return tokenized_inputs





# %%
# Convert pandas DataFrame to Hugging Face's Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(val_df) 
test_dataset = Dataset.from_pandas(test_df)

# Map the tokenization function across the datasets
train_dataset = train_dataset.map(tokenize_and_format, batched=True,batch_size=16)
eval_dataset = eval_dataset.map(tokenize_and_format, batched=True,batch_size=16) 
test_dataset = test_dataset.map(tokenize_and_format, batched=True,batch_size=16)




Maximum token length: 499
Next maximum token length: 499
Average token length: 102.76


Map: 100%|██████████| 60794/60794 [00:37<00:00, 1611.77 examples/s]


In [6]:


# %%


# %%
from sklearn.metrics import f1_score, precision_score, recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'eval_f1': f1,
        'eval_precision': precision,
        'eval_recall': recall,
    }

 

 
 

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Use 'epoch' to evaluate at the end of each epoch
    save_strategy="epoch",  # Also use 'epoch' to save at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model at the end of training based on metric
    metric_for_best_model='f1',  # Define the metric for evaluating the best model
    logging_dir='./logs',
    logging_steps=10,
    report_to=[] 
)


 

trainer = Trainer(
    model=model,
    args=training_args ,  # Here you will need to make sure that the Trainer is set up correctly
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

 




/home/bhairavi/om/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:

 
io=   pd.DataFrame(eval_dataset)
 
batch_size = 4

# Convert your DataFrame to a list of texts
texts = io['text'].tolist()

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the specified device
model = model.to(device)
 
model.eval()

# Disable gradient calculations for efficiency
with torch.no_grad():
    # Initialize lists to store predictions
    all_predictions = []
    
    # Process data in batches
    for start in range(0, len(texts), batch_size):
        end = start + batch_size
        batch_texts = texts[start:end]
        
        # Tokenize the batch
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        
        # Move inputs to the same device as model
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Get predictions
        outputs = model(**inputs)
        # Apply softmax to get probabilities from the logits
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        # Get the predicted class indices
        predicted_indices = torch.argmax(probabilities, dim=1)
        
        # Ensure the indices are back on CPU for DataFrame operations
        predicted_indices = predicted_indices.cpu()
        
        # Convert indices to actual labels if labels are strings
        if hasattr(tokenizer, 'get_labels') and callable(tokenizer.get_labels):
            predicted_labels = [tokenizer.get_labels()[idx] for idx in predicted_indices]
        else:
            predicted_labels = predicted_indices.tolist()
        
        # Append batch predictions to the list
        all_predictions.extend(predicted_labels)


io['predicted_label'] = all_predictions
 


In [8]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

top_n = 5


In [9]:
io[f'significant_{ngram_length}grams'] = None
io[f'{ngram_length}gram_weights'] = None 

def occlusion(text, model, tokenizer, label, ngram_length, device):
    inputs = tokenizer(text, return_tensors='pt').to(device)
    original_logits = model(**inputs).logits
    original_probs = F.softmax(original_logits, dim=-1)
    original_prediction = original_probs[0][label].item()
    ngram_importances = defaultdict(float)
    words = text.split()
    ngrams_list = list(ngrams(words, ngram_length))
    for ngram in ngrams_list:
        occluded_text = " ".join([word if word not in ngram else "[OCCLUDED]" for word in words])
        occluded_inputs = tokenizer(occluded_text, return_tensors='pt').to(device)
        occluded_logits = model(**occluded_inputs).logits
        occluded_probs = F.softmax(occluded_logits, dim=-1)
        occluded_prediction = occluded_probs[0][label].item()
        ngram_importances[" ".join(ngram)] = original_prediction - occluded_prediction
        
    return ngram_importances

def aggregate_and_filter_positive_attributions(ngram_attributions, threshold=0):
    significant_ngrams = {
        ngram: value for ngram, value in ngram_attributions.items()
        if value > threshold
    }
    return significant_ngrams

def select_top_ngrams(ngram_importances, top_n=top_n):
    sorted_ngrams = sorted(ngram_importances.items(), key=lambda item: item[1], reverse=True)
    top_ngrams = sorted_ngrams[:top_n]
    return top_ngrams

for index, row in io.iterrows():
    if row['target'] == row['predicted_label']:  
        ngram_attributions = occlusion(row['text'], model, tokenizer, row['label'], ngram_length, device)
        positive_attributions = aggregate_and_filter_positive_attributions(ngram_attributions)
        top_ngrams = select_top_ngrams(positive_attributions, top_n=top_n)  
        io.at[index, f'significant_{ngram_length}grams'] = [ngram for ngram, _ in top_ngrams]  
        io.at[index, f'{ngram_length}gram_weights'] = [weight for _, weight in top_ngrams]  


In [ ]:

io.to_csv(filepath_full,index= False)

io[0:4]

,Unnamed: 0,label,text,target,__index_level_0__,input_ids,token_type_ids,attention_mask,predicted_label,significant_3grams,3gram_weights
0,205,10,"My skin has developed rashes, mainly near the ...",10,205,"[1, 573, 1158, 303, 1454, 44216, 261, 3325, 94...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",10,"[rash is spreading, rashes, mainly near, has d...","[0.7546083182096481, 0.7232965230941772, 0.680..."
1,62,5,"I keep sneezing, and I'm miserable and exhaust...",5,362,"[1, 273, 548, 51653, 261, 263, 273, 280, 358, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5,"[sneezing, and I'm, I keep sneezing,, keep sne...","[0.022240877151489258, 0.01738584041595459, 0...."
2,11,8,"Doctor, I have a really itchy rash on my skin ...",8,311,"[1, 6244, 261, 273, 286, 266, 431, 29879, 1628...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",8,"[skin and there, on my skin, my skin and, spot...","[0.00855487585067749, 0.0047209858894348145, 0..."
3,30,15,"Because of dry, flaky areas on my skin, I am p...",15,30,"[1, 1665, 265, 1966, 261, 40341, 893, 277, 312...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",15,"[knees and elbows, flaky areas on, dry, flaky ...","[0.031492531299591064, 0.023659825325012207, 0..."


In [ ]:
io[f'significant_{ngram_length}grams'][0]

['rash is spreading',
 'rashes, mainly near',
 'has developed rashes,',
 'developed rashes, mainly',
 'nose. The rash']

In [ ]:


# %%

 
io['text'][0], io[f'significant_{ngram_length}grams'][0]



('My skin has developed rashes, mainly near the nose. The rash is spreading to other parts of my body. I am really about worried about this.',
 ['rash is spreading',
  'rashes, mainly near',
  'has developed rashes,',
  'developed rashes, mainly',
  'nose. The rash'])

In [ ]:

# %%

# %%
io['label'] = le.inverse_transform(io['target'])

 


In [ ]:


# %%
io= io.dropna()

# %%


In [ ]:
# label_to_words = io.groupby('label')['significant_words'].apply(lambda words: set().union(*words)).to_dict()
label_to_words = io.groupby('label')[f'significant_{ngram_length}grams'].apply(lambda words: set().union(*words)).to_dict()
# label_to_words = io.groupby('label')['significant_5grams'].apply(lambda words: set().union(*words)).to_dict()
 
print(label_to_words)



{'Acne': {'are many pus-filled', 'riddled with blackheads', 'Blackheads and pus-filled', 'pimples and blackheads', 'filled with pus.', 'several blackheads and', 'pimples and blackheads.', 'with pus. My', 'and blackheads all', 'pus-filled pimples have', 'blackheads and pus-filled', 'Blackheads and pimples', 'has recently been', 'loaded with pus.', 'and pus-filled pimples.', 'with blackheads and', 'pimples have blocked', 'Additionally, my skin', 'of blackheads and', 'and pimples filled', 'skin rash. Blackheads', 'My skin has', 'and pus-filled pimples', 'pus-filled pimples and', 'blackheads and pimples', 'pus-filled pimples. My', 'skin has been', 'blackheads. My skin', 'tons of blackheads', 'many pus-filled pimples', 'been scurrying about.', 'are several blackheads'}, 'Arthritis': {'joints. Walking has', 'neck has been', 'Recently, when I', 'stiff. additionally, walking', 'move. Walking has', 'joints have been', 'to move. Walking', 'Since my joints', 'has been extremely', 'stiff. My joint

In [ ]:

# %%

# %%



label_to_words_and_weights = {}

for label, group in io.groupby('label'):
    word_weights_dict = {}
    for index, row in group.iterrows():
        words = row[f'significant_{ngram_length}grams']
        weights = row[f'{ngram_length}gram_weights']
        for word, weight in zip(words, weights):
            if word in word_weights_dict:
                # Take the maximum of the existing and current weight
                word_weights_dict[word] = max(word_weights_dict[word], weight)
            else:
                word_weights_dict[word] = weight
    label_to_words_and_weights[label] = word_weights_dict

# Display the dictionary with labels, words, and their maximum weights
print(label_to_words_and_weights)


{'Acne': {'blackheads. My skin': 0.11732691526412964, 'My skin has': 0.07348406314849854, 'pimples and blackheads.': 0.05616527795791626, 'pus-filled pimples and': 0.16073071956634521, 'skin has been': 0.12482476234436035, 'blackheads and pimples': 0.2121880054473877, 'several blackheads and': 0.05635547637939453, 'loaded with pus.': 0.03396618366241455, 'are several blackheads': 0.03138214349746704, 'been scurrying about.': 0.024169862270355225, 'blackheads and pus-filled': 0.3756505250930786, 'pus-filled pimples. My': 0.05431830883026123, 'and pus-filled pimples.': 0.05159181356430054, 'with blackheads and': 0.02048647403717041, 'riddled with blackheads': 0.01522839069366455, 'and pus-filled pimples': 0.7401902824640274, 'pus-filled pimples have': 0.6890276670455933, 'Blackheads and pus-filled': 0.4842521548271179, 'pimples have blocked': 0.17955631017684937, 'has recently been': 0.08096969127655029, 'tons of blackheads': 0.07971066236495972, 'of blackheads and': 0.05250990390777588,

In [ ]:

# %%


# %%
label_to_words = label_to_words_and_weights

# %%
len(list(label_to_words['Acne']))



32

In [ ]:

# %%

# %%
len(label_to_words)



24

In [ ]:

# %%

# %%
label_to_words

 


{'Acne': {'blackheads. My skin': 0.11732691526412964,
  'My skin has': 0.07348406314849854,
  'pimples and blackheads.': 0.05616527795791626,
  'pus-filled pimples and': 0.16073071956634521,
  'skin has been': 0.12482476234436035,
  'blackheads and pimples': 0.2121880054473877,
  'several blackheads and': 0.05635547637939453,
  'loaded with pus.': 0.03396618366241455,
  'are several blackheads': 0.03138214349746704,
  'been scurrying about.': 0.024169862270355225,
  'blackheads and pus-filled': 0.3756505250930786,
  'pus-filled pimples. My': 0.05431830883026123,
  'and pus-filled pimples.': 0.05159181356430054,
  'with blackheads and': 0.02048647403717041,
  'riddled with blackheads': 0.01522839069366455,
  'and pus-filled pimples': 0.7401902824640274,
  'pus-filled pimples have': 0.6890276670455933,
  'Blackheads and pus-filled': 0.4842521548271179,
  'pimples have blocked': 0.17955631017684937,
  'has recently been': 0.08096969127655029,
  'tons of blackheads': 0.07971066236495972,
 

In [ ]:

test_data = pd.DataFrame(test_dataset)

 

 
from nltk.tokenize import sent_tokenize, word_tokenize

# %%
def split_text(text):
 
    sentences = sent_tokenize(text)  # Tokenize the text into sentences
    third_index = len(sentences) // 2  # Calculate index for one-third
    first_part = ' '.join(sentences[:third_index])  # First third of sentences
    remaining_part = ' '.join(sentences[third_index:])  # Remaining two-thirds
    return first_part, remaining_part
 
# Apply the function to the DataFrame
test_data[['first_half', 'second_half']] = test_data['text'].apply(lambda x: pd.Series(split_text(x)))


# %%
test_data['first_half'][0:4] , test_data['second_half'][0:4]

# %%

 
# %%


(0                    My body is shaking and trembling.
 1    I can't seem to catch my breath and I'm sweati...
 2    The veins on my legs are causing a lot of disc...
 3    I have a high fever and red spots and rashes a...
 Name: first_half, dtype: object,
 0    My senses of taste and smell have gotten weake...
 1    My chest hurts and my heart is racing. The muc...
 2    They are swollen and protruding from my skin, ...
 3    I feel exhausted and have completely lost my a...
 Name: second_half, dtype: object)

In [ ]:

# %%
tdf = test_data[['first_half', 'label']]
tdf = Dataset.from_pandas(tdf)


 
   
def tokenize_function(examples):
    return tokenizer(examples['first_half'], truncation=True, padding="max_length", max_length=512)

tdf = tdf.map(tokenize_function, batched=True)

# Predict using the trained model
output = trainer.predict(tdf)
predictions = np.argmax(output.predictions, axis=1)
labels = output.label_ids 

# Generate the classification report
report = classification_report(
    labels,
    predictions,
    target_names=np.unique(labels).astype(str),  # Convert labels to string if necessary
    digits=4
)
print(Fore.RED +"first half i.e partial info classification report, baseline 1")
print(report)



Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map: 100%|██████████| 60/60 [00:00<00:00, 1890.89 examples/s]


/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


first half i.e partial info classification report, baseline 1
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         2
           1     1.0000    1.0000    1.0000         2
           2     1.0000    0.5000    0.6667         2
           3     1.0000    0.6667    0.8000         3
           4     0.5000    1.0000    0.6667         3
           5     0.3333    1.0000    0.5000         2
           6     1.0000    0.6667    0.8000         3
           7     0.4000    0.6667    0.5000         3
           8     1.0000    1.0000    1.0000         2
           9     1.0000    0.5000    0.6667         2
          10     0.3333    0.3333    0.3333         3
          11     1.0000    1.0000    1.0000         2
          12     1.0000    1.0000    1.0000         2
          13     0.0000    0.0000    0.0000         2
          14     0.0000    0.0000    0.0000         3
          15     1.0000    0.6667    0.8000         3
          16     1.

/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Move model to the specified device
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Disable gradient calculations for efficiency
with torch.no_grad():
    # Assuming your DataFrame `io` has a column 'text' containing the text to predict
    texts = test_data['first_half'].tolist()
    # Tokenize the text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Move inputs to the same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get predictions
    outputs = model(**inputs)
    # Apply softmax to get probabilities from the logits
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    # Get the top 3 predicted class indices and their probabilities
    top_probs, top_indices = torch.topk(probabilities, 3, dim=1)

    # Ensure the indices and probabilities are back on CPU for DataFrame operations
    top_indices = top_indices.cpu()
    top_probs = top_probs.cpu()

    # Convert indices to actual labels if labels are strings
    if hasattr(tokenizer, 'get_labels') and callable(tokenizer.get_labels):
        predicted_labels = [[tokenizer.get_labels()[idx] for idx in indices] for indices in top_indices]
    else:
        predicted_labels = top_indices.tolist()

    # Add top 3 predictions to DataFrame
    test_data['top3_predicted_labels'] = predicted_labels
    test_data['top3_predicted_probabilities'] = top_probs.tolist()


 
test_data = test_data[['text','label','target','first_half','second_half','top3_predicted_labels']]

 
 

# %%
test_data['top3_predicted_target']  =  test_data['top3_predicted_labels']

# %%
test_data['label'] = le.inverse_transform(test_data['target'])

# Decode 'top3_predicted_labels' - since these are lists, we apply the inverse transform in a vectorized manner
test_data['top3_predicted_labels'] = test_data['top3_predicted_labels'].apply(lambda x: le.inverse_transform(x))

 

# %%

# %%
def map_significant_words(predictions):
    print(predictions)
    for row in predictions:
        print(row)
    return [label_to_words.get(row, []) for row in predictions]
    # return [[result[pred] for pred in row] for row in predictions]

# Applying the mapping function
test_data['significant_words'] = test_data['top3_predicted_labels'].apply(map_significant_words)

# %%
test_data


['Varicose Veins' 'allergy' 'Chicken pox']
Varicose Veins
allergy
Chicken pox
['Common Cold' 'Pneumonia' 'allergy']
Common Cold
Pneumonia
allergy
['Varicose Veins' 'Chicken pox' 'drug reaction']
Varicose Veins
Chicken pox
drug reaction
['Chicken pox' 'Dengue' 'Impetigo']
Chicken pox
Dengue
Impetigo
['Pneumonia' 'Bronchial Asthma' 'Common Cold']
Pneumonia
Bronchial Asthma
Common Cold
['Jaundice' 'Malaria' 'Hypertension']
Jaundice
Malaria
Hypertension
['Chicken pox' 'Dengue' 'Typhoid']
Chicken pox
Dengue
Typhoid
['Dimorphic Hemorrhoids' 'Impetigo' 'Bronchial Asthma']
Dimorphic Hemorrhoids
Impetigo
Bronchial Asthma
['urinary tract infection' 'peptic ulcer disease' 'Psoriasis']
urinary tract infection
peptic ulcer disease
Psoriasis
['Dengue' 'Typhoid' 'Chicken pox']
Dengue
Typhoid
Chicken pox
['Malaria' 'Dengue' 'Typhoid']
Malaria
Dengue
Typhoid
['Varicose Veins' 'Chicken pox' 'Dengue']
Varicose Veins
Chicken pox
Dengue
['Dimorphic Hemorrhoids' 'Impetigo' 'Bronchial Asthma']
Dimorphic Hemo

,text,label,target,first_half,second_half,top3_predicted_labels,top3_predicted_target,significant_words
0,My body is shaking and trembling. My senses of...,diabetes,19,My body is shaking and trembling.,My senses of taste and smell have gotten weake...,"[Varicose Veins, allergy, Chicken pox]","[17, 18, 4]","[{'in my legs': 0.488895982503891, 'my legs ar..."
1,I can't seem to catch my breath and I'm sweati...,Pneumonia,14,I can't seem to catch my breath and I'm sweati...,My chest hurts and my heart is racing. The muc...,"[Common Cold, Pneumonia, allergy]","[5, 14, 18]","[{'sneezing, and I'm': 0.032268226146698, 'I k..."
2,The veins on my legs are causing a lot of disc...,Varicose Veins,17,The veins on my legs are causing a lot of disc...,"They are swollen and protruding from my skin, ...","[Varicose Veins, Chicken pox, drug reaction]","[17, 4, 20]","[{'in my legs': 0.488895982503891, 'my legs ar..."
3,I have a high fever and red spots and rashes a...,Chicken pox,4,I have a high fever and red spots and rashes a...,I feel exhausted and have completely lost my a...,"[Chicken pox, Dengue, Impetigo]","[4, 6, 10]","[{'swollen, causing discomfort': 0.48434475064..."
4,"I have been feeling extremely tired and weak, ...",Bronchial Asthma,2,"I have been feeling extremely tired and weak, ...","My fever is very high, and I'm producing a lot...","[Pneumonia, Bronchial Asthma, Common Cold]","[14, 2, 5]","[{'some brownish phlegm.': 0.9015878848731518,..."
5,"I've been feeling extremely scratchy, sick, an...",Jaundice,11,"I've been feeling extremely scratchy, sick, an...","My urine is dark, and my skin has turned yello...","[Jaundice, Malaria, Hypertension]","[11, 12, 9]","[{'dark urine, and': 0.04527640342712402, 'hav..."
6,I've had trouble sleeping because of the itchi...,Psoriasis,15,I've had trouble sleeping because of the itchi...,"My nails have little dents, which is really al...","[Chicken pox, Dengue, Typhoid]","[4, 6, 16]","[{'swollen, causing discomfort': 0.48434475064..."
7,"I've been grumpy and gloomy lately, and I've a...",Migraine,13,,"I've been grumpy and gloomy lately, and I've a...","[Dimorphic Hemorrhoids, Impetigo, Bronchial As...","[7, 10, 2]","[{'anus has also': 0.03569042682647705, 'My an..."
8,"My lower abdomen hurts, and when I urinate, it...",urinary tract infection,23,"My lower abdomen hurts, and when I urinate, it...",I've been having low body temps ever since I s...,"[urinary tract infection, peptic ulcer disease...","[23, 22, 15]","[{'intense, uncontrollable urges': 0.012143015..."
9,The nausea I have been feeling is accompanied ...,Dengue,6,The nausea I have been feeling is accompanied ...,There is a distinct pain in my back and muscle...,"[Dengue, Typhoid, Chicken pox]","[6, 16, 4]","[{'pain and vomitting.': 0.42897385358810425, ..."


In [ ]:


# %%

# %%
test_data.to_csv(file_path,index= False)


In [ ]:

# %%
df = test_data

 
df['significant_words'][0]

 


[{'in my legs': 0.488895982503891,
  'my legs are': 0.4594966769218445,
  'veins in my': 0.2333756685256958,
  'legs are causing': 0.23197436332702637,
  'The veins in': 0.08885067701339722,
  'on my legs': 0.17966949939727783,
  'legs that I': 0.07607412338256836,
  'vessels below the': 0.06379151344299316,
  'my legs that': 0.05489623546600342,
  'There is bruising': 0.02944272756576538,
  "My legs' skin": 0.09659463167190552,
  'skin around the': 0.06327491998672485,
  "legs' skin around": 0.043838322162628174,
  'veins is unusual.': 0.042717158794403076,
  'around the veins': 0.036156415939331055,
  'in my calves': 0.9620735943317413,
  'my calves are': 0.3880428671836853,
  'calves are making': 0.24857032299041748,
  'it difficult for': 0.10371333360671997,
  'difficult for me': 0.06708598136901855,
  'The veins on': 0.16718816757202148,
  'veins on my': 0.12512195110321045,
  'lot of discomforts': 0.11622029542922974,
  'of discomforts when': 0.08438414335250854,
  'my calves is'

In [ ]:


# %%
# %%
df.rename(columns={'significant_words': 'significant_words_weights'}, inplace=True)
 


In [ ]:
# %%
len(df['significant_words_weights'][0])


3

In [ ]:

# %%
df['significant_words_weights'][0][0]  

# %%


{'in my legs': 0.488895982503891,
 'my legs are': 0.4594966769218445,
 'veins in my': 0.2333756685256958,
 'legs are causing': 0.23197436332702637,
 'The veins in': 0.08885067701339722,
 'on my legs': 0.17966949939727783,
 'legs that I': 0.07607412338256836,
 'vessels below the': 0.06379151344299316,
 'my legs that': 0.05489623546600342,
 'There is bruising': 0.02944272756576538,
 "My legs' skin": 0.09659463167190552,
 'skin around the': 0.06327491998672485,
 "legs' skin around": 0.043838322162628174,
 'veins is unusual.': 0.042717158794403076,
 'around the veins': 0.036156415939331055,
 'in my calves': 0.9620735943317413,
 'my calves are': 0.3880428671836853,
 'calves are making': 0.24857032299041748,
 'it difficult for': 0.10371333360671997,
 'difficult for me': 0.06708598136901855,
 'The veins on': 0.16718816757202148,
 'veins on my': 0.12512195110321045,
 'lot of discomforts': 0.11622029542922974,
 'of discomforts when': 0.08438414335250854,
 'my calves is': 0.4414995312690735,
 'l

In [ ]:
top_n = 15  # The number of top words you want to select

def filter_top_n_words(list_of_dicts):
    processed_list = []
    
    for index, dct in enumerate(list_of_dicts):
        if isinstance(dct, dict):
            # Sort the items by value in descending order
            sorted_items = sorted(dct.items(), key=lambda item: item[1], reverse=True)
            # Select the top N words
            top_items = sorted_items[:top_n]
            # Extract the keys (words) and add them to the list as a set
            processed_list.append(set(key for key, value in top_items))
        else:
            # Print the index and the problematic element
            print(f"Non-dictionary element at index {index}: {dct}")
            # Optionally, add an empty set or handle differently based on your needs
            processed_list.append(set())
    
    return processed_list

# Apply the function to each element in the DataFrame column
df['significant_words'] = df['significant_words_weights'].apply(filter_top_n_words)


In [ ]:

# %%
df['significant_words'][0][0]  , df['significant_words'][0][1], df['significant_words'][0][2]


# %%


({'Long durations of',
  "My legs' skin",
  'The veins in',
  'The veins on',
  'There is bruising',
  'around the veins',
  'calves are making',
  'calves is constant',
  'difficult for me',
  'have caused severe',
  'in my calves',
  'in my legs',
  'it difficult for',
  'legs are causing',
  'legs that I',
  "legs' skin around",
  'long periods of',
  'lot of discomforts',
  'my calves are',
  'my calves is',
  'my legs are',
  'my legs that',
  'of discomforts when',
  'on my legs',
  'skin around the',
  'stand or walk',
  'standing or walking',
  'veins in my',
  'veins is unusual.',
  'veins on my',
  'vessels below the',
  'walking have caused',
  'worse the longer'},
 {'I get chest',
  'I have a',
  'I occasionally get',
  'I occasionally have',
  "I'm nauseous, ill,",
  'My skin has',
  'My throat always',
  'Sometimes at night',
  'always tickles, and',
  'annoying. I occasionally',
  'around my eyes',
  'breathe. I occasionally',
  'breathing. Sometimes at',
  'can occasion

In [ ]:

# %%
df.shape


# %%


(60, 9)

In [ ]:
 
df['significant_words'][0][0]  , df['significant_words_weights'][0][0] 



({'Long durations of',
  "My legs' skin",
  'The veins in',
  'The veins on',
  'There is bruising',
  'around the veins',
  'calves are making',
  'calves is constant',
  'difficult for me',
  'have caused severe',
  'in my calves',
  'in my legs',
  'it difficult for',
  'legs are causing',
  'legs that I',
  "legs' skin around",
  'long periods of',
  'lot of discomforts',
  'my calves are',
  'my calves is',
  'my legs are',
  'my legs that',
  'of discomforts when',
  'on my legs',
  'skin around the',
  'stand or walk',
  'standing or walking',
  'veins in my',
  'veins is unusual.',
  'veins on my',
  'vessels below the',
  'walking have caused',
  'worse the longer'},
 {'in my legs': 0.488895982503891,
  'my legs are': 0.4594966769218445,
  'veins in my': 0.2333756685256958,
  'legs are causing': 0.23197436332702637,
  'The veins in': 0.08885067701339722,
  'on my legs': 0.17966949939727783,
  'legs that I': 0.07607412338256836,
  'vessels below the': 0.06379151344299316,
  'my

In [ ]:

# %%
# %%
df = df.dropna(ignore_index =True)


In [ ]:
# %%
df.to_csv(file_path ,index= False)

# %%
df.columns


Index(['text', 'label', 'target', 'first_half', 'second_half',
       'top3_predicted_labels', 'top3_predicted_target',
       'significant_words_weights', 'significant_words'],
      dtype='object')

In [ ]:

# %%
len(df['significant_words'][0]),len(df['significant_words'][0][0])

# %


(3, 33)